In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from numpy import float64 as f64
import matplotlib as mpl

In [ ]:
# For interactive plots
%matplotlib widget
%cd '/home/jovyan/work/labs/lab-4.2'

### Utilities

In [ ]:
label_font = 12
markersize = 8

### Data analysis

In [ ]:
df = pd.read_csv(f"data/data.csv")

df = df.drop('N, 1/s', axis=1)
df = df.drop('(N)^0.5', axis=1)
df = df.drop('(N)^0.5/I', axis=1)

df['n, 1/s'] = df['N'] / df['t, s']
df['delta_n, 1/s'] = 1.0 / np.sqrt(df['N']) * df['n, 1/s']

first_row = df.iloc[0]
n0 = first_row['n, 1/s']
delta_n0 = first_row['delta_n, 1/s']
current_delta = 0.01 # 10 mA
print(f'n0 = {n0} += {delta_n0}')

df['n, 1/s'] -= n0

In [ ]:
def plot_unfitted(ax):
    x = df['p, keV/ c']
    y = df['n, 1/s']
    dy = df['delta_n, 1/s']

    plot0 = ax.errorbar(x, y, dy, current_delta, fmt=',', capsize=2.5)
    
    ax.set_xlabel(f"$I$, A")
    ax.set_ylabel(f"$N$, 1/s")

    ax.get_xaxis().set_visible(True)
    ax.get_yaxis().set_visible(True)
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
    
    ax.grid(True, which="both")
    plt.rcParams.update({"figure.autolayout": True})

In [ ]:
def gauss(x, a, mu, sigma):
    return a * np.exp(-0.5 * (x - mu) ** 2 / sigma**2.0)
    
def fit_gaussian(xs, ys) -> tuple:
    return sp.optimize.curve_fit(gauss, xs, ys, p0=[ys.max(), xs.mean(), 1.0])

In [ ]:
slice_begin, slice_end = 16, 27
sliced = df.iloc[slice_begin:slice_end]
fit_result = fit_gaussian(sliced['I, A'], sliced['n, 1/s'])
coeff, _ = fit_result
_, mean, sigma = coeff
xs = np.linspace(mean - 3.0 * sigma, mean + 3.0 * sigma)

In [ ]:
k = 1013.5 / mean
c = 3e8 # Speed of light
mc = 511 # Photon energy
 
print(f'k = {k}')
df['p, keV/ c'] = k * df['I, A']
df['E, keV'] = np.sqrt(df['p, keV/ c'] ** 2.0 + mc ** 2.0) - mc

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

sliced = df.iloc[slice_begin:slice_end]
fit_result = fit_gaussian(sliced['p, keV/ c'], sliced['n, 1/s'])
coeff, _ = fit_result
_, mean, sigma = coeff
xs = np.linspace(mean - 3.0 * sigma, mean + 3.0 * sigma)
plot_unfitted(ax)
ax.plot(xs, np.vectorize(gauss)(xs, *coeff))

fig.savefig(f"output/current-without-fit.pdf")

In [ ]:
def plot_curie(ax):
    x = df['E, keV']
    y = df['n, 1/s'] ** 0.5 * df['p, keV/ c'] ** -1.5

    plot0 = ax.scatter(x, y)
    
    ax.set_xlabel(f"$T$, keV")
    ax.set_ylabel(f"$f$")

    ax.get_xaxis().set_visible(True)
    ax.get_yaxis().set_visible(True)
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
    
    ax.grid(True, which="both")
    plt.rcParams.update({"figure.autolayout": True})

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

plot_curie(ax)

sliced = df[8:17]
x = sliced['E, keV']
y = sliced['n, 1/s'] ** 0.5 * sliced['p, keV/ c'] ** -1.5

result = sp.stats.linregress(x, y)
xs = np.linspace(min(df['E, keV']), max(df['E, keV']), 1000)
ax.plot(xs, xs * result.slope + result.intercept)

print(f'e_max = {-result.intercept / result.slope}')
fig.savefig(f"output/curie.pdf")